# Identify problem type

> This notebook works in Google Colab. It gets data from kaggle's homesite-quote-conversion, cleans dates and ints stored as str, checks quote date in training data and test data to see if time series

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- author: Tim Cummings


Install packages recommended in fastbook Ch09

In [1]:
!pip install -Uqq fastbook kaggle waterfallcharts treeinterpreter dtreeviz

     |████████████████████████████████| 727kB 25.5MB/s 
     |████████████████████████████████| 61kB 10.1MB/s 
     |████████████████████████████████| 194kB 41.2MB/s 
     |████████████████████████████████| 1.2MB 48.1MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████| 61kB 10.2MB/s 


In [2]:
import fastbook
fastbook.setup_book()

Mounted at /content/gdrive


In [3]:
from fastbook import *
from fastai.vision.widgets import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

Upload your kaggle.json API key

In [4]:
btn_upload = widgets.FileUpload(description="kaggle.json")
btn_upload

FileUpload(value={}, description='kaggle.json')

Save credentials

In [5]:
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.parent.exists():
    cred_path.parent.mkdir()
if len(btn_upload.data) > 0:
    with open(cred_path, mode="wb") as cred_file:
        cred_file.write(btn_upload.data[-1])
cred_path.chmod(0o600)

In [6]:
from kaggle import api

Note that '!pip install kaggle' does not update cli kaggle in Google colab and is only v1.5.4 while kaggle.api is v1.5.12

In [8]:
!kaggle --version

Kaggle API 1.5.4


Python's `kaggle.api` is using a more recent version

In [9]:
api.__version__

'1.5.12'

Get data from kaggle, extract and store in _data

In [10]:

path_hqc = (Path.cwd()/"_data")
path_hqc.mkdir(exist_ok=True)
Path.BASE_PATH = path_hqc
api.competition_download_cli('homesite-quote-conversion', path=path_hqc)
file_extract(path_hqc/"homesite-quote-conversion.zip")
file_extract(path_hqc/"train.csv.zip")
file_extract(path_hqc/"test.csv.zip")

 29%|██▉       | 18.0M/62.0M [00:00<00:00, 184MB/s]

100%|██████████| 62.0M/62.0M [00:00<00:00, 259MB/s]


Check what the data looks like

In [12]:
df = pd.read_csv(path_hqc/"train.csv", low_memory=False)
df.head()

,QuoteNumber,Original_Quote_Date,QuoteConversion_Flag,Field6,...,GeographicField62A,GeographicField62B,GeographicField63,GeographicField64
0,1,2013-08-16,0,B,...,-1,10,N,CA
1,2,2014-04-22,0,F,...,-1,20,N,NJ
2,4,2014-08-25,0,F,...,-1,8,N,NJ
3,6,2013-04-15,0,J,...,-1,21,N,TX
4,8,2014-01-25,0,E,...,-1,12,N,IL


Check how much data we have and check if `QuoteNumber` is unique

In [13]:
df.shape, len(df['QuoteNumber'].unique())


((260753, 299), 260753)

Conclusion: `QuoteNumber` ***is*** unique

We don't want to use `QuoteNumber` as a feature but we could use it as the index

In [ ]:
df = df.set_index('QuoteNumber')

Examine data types in `train.csv`

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260753 entries, 0 to 260752
Columns: 299 entries, QuoteNumber to GeographicField64
dtypes: float64(6), int64(265), object(28)
memory usage: 594.8+ MB


Find the 28 fields which do not have numeric datatypes

In [15]:
from collections import defaultdict
dct_fields_by_dtype = defaultdict(list)
for i, dt in enumerate(df.dtypes):
    dct_fields_by_dtype[dt].append(df.dtypes.index[i])
print("dtypes in train.csv:", dct_fields_by_dtype.keys())
print("fields for object dtype:", dct_fields_by_dtype[np.dtype('O')])
print("number of fields of object dtype:", len(dct_fields_by_dtype[np.dtype('O')]))

dtypes in train.csv: dict_keys([dtype('int64'), dtype('O'), dtype('float64')])
fields for object dtype: ['Original_Quote_Date', 'Field6', 'Field10', 'Field12', 'CoverageField8', 'CoverageField9', 'SalesField7', 'PersonalField7', 'PersonalField16', 'PersonalField17', 'PersonalField18', 'PersonalField19', 'PropertyField3', 'PropertyField4', 'PropertyField5', 'PropertyField7', 'PropertyField14', 'PropertyField28', 'PropertyField30', 'PropertyField31', 'PropertyField32', 'PropertyField33', 'PropertyField34', 'PropertyField36', 'PropertyField37', 'PropertyField38', 'GeographicField63', 'GeographicField64']
number of fields of object dtype: 28


`Original_Quote_Date` can be converted to datetime

In [ ]:
 df['Original_Quote_Date'] = pd.to_datetime(df['Original_Quote_Date'])
# 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 260753 entries, 1 to 434588
Columns: 298 entries, Original_Quote_Date to GeographicField64
dtypes: datetime64[ns](1), float64(6), int64(264), object(27)
memory usage: 594.8+ MB


Recalculate breakdown now that we have changed dtype of `Original_Quote_Date`

In [16]:
dct_fields_by_dtype = defaultdict(list)
for i, dt in enumerate(df.dtypes):
    dct_fields_by_dtype[dt].append(df.dtypes.index[i])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260753 entries, 0 to 260752
Columns: 299 entries, QuoteNumber to GeographicField64
dtypes: float64(6), int64(265), object(28)
memory usage: 594.8+ MB


Compare `Original_Quote_Date` in `train.csv` and `test.csv`

In [ ]:
df_test = pd.read_csv(path_hqc/"test.csv", low_memory=False)
df_test["Original_Quote_Date"] = pd.to_datetime(df_test["Original_Quote_Date"])
print("train.csv", df['Original_Quote_Date'].min(), df['Original_Quote_Date'].max(), df.shape)
print("test.csv ", df_test['Original_Quote_Date'].min(), df_test['Original_Quote_Date'].max(), df_test.shape)

train.csv 2013-01-01 00:00:00 2015-05-18 00:00:00 (260753, 298)
test.csv  2013-01-01 00:00:00 2015-05-18 00:00:00 (173836, 298)


Conclusion: overlapping date ranges (in fact identical date ranges) so don't need to consider as time series problem 

Check the non-numeric values in other object fields

In [17]:
for col in dct_fields_by_dtype[np.dtype('O')]:
    print(f"{col:20s} {df[col].unique()}")

Original_Quote_Date  ['2013-08-16' '2014-04-22' '2014-08-25' '2013-04-15' '2014-01-25' '2014-01-18' '2013-11-01' '2014-05-14' '2013-06-19' '2013-06-18' '2013-09-24' '2013-06-11' '2013-10-22' '2013-08-20' '2015-02-24'
 '2014-10-27' '2015-02-06' '2013-10-02' '2014-09-15' '2014-03-18' '2014-10-15' '2014-04-03' '2015-03-31' '2013-05-30' '2015-03-30' '2013-09-17' '2015-03-07' '2013-06-06' '2013-01-22' '2013-04-16'
 '2014-03-04' '2014-02-24' '2015-05-12' '2014-02-01' '2013-08-12' '2015-05-13' '2014-04-13' '2014-05-02' '2015-05-18' '2015-01-28' '2013-12-30' '2014-06-20' '2014-01-08' '2014-10-08' '2013-07-26'
 '2013-07-11' '2014-12-15' '2014-12-08' '2015-02-22' '2013-09-19' '2015-05-07' '2015-01-14' '2013-04-22' '2014-05-29' '2014-02-12' '2014-06-02' '2014-05-18' '2014-01-20' '2013-07-16' '2013-02-15'
 '2015-03-11' '2015-03-12' '2014-07-30' '2013-08-23' '2013-08-30' '2013-04-03' '2013-06-20' '2015-01-25' '2013-12-10' '2015-01-05' '2013-12-03' '2014-11-18' '2013-07-29' '2013-12-25' '2015-01-16'

`Field10` looks like integers stored as strings so convert to `int`s

In [18]:
df['Field10'] = df['Field10'].str.replace(",", "").astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260753 entries, 0 to 260752
Columns: 299 entries, QuoteNumber to GeographicField64
dtypes: float64(6), int64(266), object(27)
memory usage: 594.8+ MB


Eecalculate breakdown now that we have changed dtype of `Field10`

In [19]:
dct_fields_by_dtype = defaultdict(list)
for i, dt in enumerate(df.dtypes):
    dct_fields_by_dtype[dt].append(df.dtypes.index[i])